# How to count tokens with tiktoken
# 怎么用tiktoken计算令牌数

[`tiktoken`](https://github.com/openai/tiktoken/blob/main/README.md) is a fast open-source tokenizer by OpenAI.
[`tiktoken`](https://github.com/openai/tiktoken/blob/main/README.md) 是OpenAI的一个快速开源分词器。
Given a text string (e.g., `"tiktoken is great!"`) and an encoding (e.g., `"cl100k_base"`), a tokenizer can split the text string into a list of tokens (e.g., `["t", "ik", "token", " is", " great", "!"]`).
给定一个文本字符串（例如，`"tiktoken is great!"`）和一个编码（例如，`"cl100k_base"`），一个分词器可以将文本字符串分割为令牌列表（例如，`["t", "ik", "token", " is", " great", "!"]`）。
Splitting text strings into tokens is useful because GPT models see text in the form of tokens. Knowing how many tokens are in a text string can tell you (a) whether the string is too long for a text model to process and (b) how much an OpenAI API call costs (as usage is priced by token).
将文本字符串分割为令牌是有用的，因为GPT模型以令牌的形式查看文本。知道文本字符串中有多少令牌可以告诉你（a）字符串是否对于文本模型来说太长了，无法处理（b）OpenAI API调用的成本（因为使用是按令牌计价的）。

## Encodings
## 编码
Encodings specify how text is converted into tokens. Different models use different encodings.
编码指定如何将文本转换为令牌。不同的模型使用不同的编码。
`tiktoken` supports three encodings used by OpenAI models:
`tiktoken` 支持OpenAI模型使用的三种编码：

| Encoding name           | OpenAI models                                       |
|-------------------------|-----------------------------------------------------|
| `cl100k_base`           | `gpt-4`, `gpt-3.5-turbo`, `text-embedding-ada-002`  |
| `p50k_base`             | Codex models, `text-davinci-002`, `text-davinci-003`|
| `r50k_base` (or `gpt2`) | GPT-3 models like `davinci`                         |

You can retrieve the encoding for a model using `tiktoken.encoding_for_model()` as follows:
你可以使用`tiktoken.encoding_for_model()`检索模型的编码，如下所示：
```python
encoding = tiktoken.encoding_for_model('gpt-3.5-turbo')
```

Note that `p50k_base` overlaps substantially with `r50k_base`, and for non-code applications, they will usually give the same tokens.
请注意，`p50k_base`与`r50k_base`有很大的重叠，对于非代码应用程序，它们通常会产生相同的令牌。
## Tokenizer libraries by language
## 语言的分词器库
For `cl100k_base` and `p50k_base` encodings:
对于`cl100k_base`和`p50k_base`编码：
- Python: [tiktoken](https://github.com/openai/tiktoken/blob/main/README.md)
- .NET / C#: [SharpToken](https://github.com/dmitry-brazhenko/SharpToken)

For `r50k_base` (`gpt2`) encodings, tokenizers are available in many languages.
对于`r50k_base`（`gpt2`）编码，分词器可用于许多语言。
- Python: [tiktoken](https://github.com/openai/tiktoken/blob/main/README.md) (or alternatively [GPT2TokenizerFast](https://huggingface.co/docs/transformers/model_doc/gpt2#transformers.GPT2TokenizerFast))
- JavaScript: [gpt-3-encoder](https://www.npmjs.com/package/gpt-3-encoder)
- .NET / C#: [GPT Tokenizer](https://github.com/dluc/openai-tools)
- Java: [gpt2-tokenizer-java](https://github.com/hyunwoongko/gpt2-tokenizer-java)
- PHP: [GPT-3-Encoder-PHP](https://github.com/CodeRevolutionPlugins/GPT-3-Encoder-PHP)

(OpenAI makes no endorsements or guarantees of third-party libraries.)
（OpenAI不对第三方库做任何认可或保证。）

## How strings are typically tokenized
## 通常如何分词字符串
In English, tokens commonly range in length from one character to one word (e.g., `"t"` or `" great"`), though in some languages tokens can be shorter than one character or longer than one word. Spaces are usually grouped with the starts of words (e.g., `" is"` instead of `"is "` or `" "`+`"is"`). You can quickly check how a string is tokenized at the [OpenAI Tokenizer](https://beta.openai.com/tokenizer).
在英语中，令牌通常在长度上从一个字符到一个单词（例如，`"t"`或`" great"`），尽管在某些语言中，令牌可以短于一个字符或长于一个单词。空格通常与单词的开始一起分组（例如，`" is"`而不是`"is "`或`" "`+`"is"`）。您可以在[OpenAI Tokenizer](https://beta.openai.com/tokenizer)上快速检查字符串如何分词。

## 0. Install `tiktoken`
## 0. 安装`tiktoken`

If needed, install `tiktoken` with `pip`:
如果需要，使用`pip`安装`tiktoken`：

In [1]:
%pip install --upgrade tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 2.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 2.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


## 1. Import `tiktoken`
## 1. 导入`tiktoken`

In [2]:
import tiktoken


## 2. Load an encoding
## 2. 加载编码

Use `tiktoken.get_encoding()` to load an encoding by name.
使用`tiktoken.get_encoding()`按名称加载编码。

The first time this runs, it will require an internet connection to download. Later runs won't need an internet connection.
第一次运行时，它将需要互联网连接以下载。以后的运行不需要互联网连接。

In [5]:
encoding = tiktoken.get_encoding("cl100k_base")


Use `tiktoken.encoding_for_model()` to automatically load the correct encoding for a given model name.
使用`tiktoken.encoding_for_model()`自动加载给定模型名称的正确编码。

In [3]:
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

## 3. Turn text into tokens with `encoding.encode()`
## 3. 使用`encoding.encode()`将文本转换为令牌

The `.encode()` method converts a text string into a list of token integers.
`.encode()`方法将文本字符串转换为令牌整数列表。

In [6]:
encoding.encode("tiktoken is great!")


[83, 1609, 5963, 374, 2294, 0]

Count tokens by counting the length of the list returned by `.encode()`.
通过计算`.encode()`返回的列表的长度来计算令牌数。

In [8]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens


In [9]:
num_tokens_from_string("tiktoken is great!", "cl100k_base")


6

## 4. Turn tokens into text with `encoding.decode()`
## 4. 使用`encoding.decode()`将令牌转换为文本

`.decode()` converts a list of token integers to a string.
`.decode()`将令牌整数列表转换为字符串。

In [10]:
encoding.decode([83, 1609, 5963, 374, 2294, 0])


'tiktoken is great!'

Warning: although `.decode()` can be applied to single tokens, beware that it can be lossy for tokens that aren't on utf-8 boundaries.
警告：尽管`.decode()`可以应用于单个令牌，但请注意，对于不在utf-8边界上的令牌，它可能会有损失。

For single tokens, `.decode_single_token_bytes()` safely converts a single integer token to the bytes it represents.
对于单个令牌，`.decode_single_token_bytes()`安全地将单个整数令牌转换为它表示的字节。

In [8]:
[encoding.decode_single_token_bytes(token) for token in [83, 1609, 5963, 374, 2294, 0]]


[b't', b'ik', b'token', b' is', b' great', b'!']

(The `b` in front of the strings indicates that the strings are byte strings.)
（字符串前面的`b`表示字符串是字节字符串。）

## 5. Comparing encodings
## 5. 比较编码

Different encodings vary in how they split words, group spaces, and handle non-English characters. Using the methods above, we can compare different encodings on a few example strings.
不同的编码在如何拆分单词、分组空格和处理非英语字符方面有所不同。使用上述方法，我们可以在几个示例字符串上比较不同的编码。

In [12]:

def compare_encodings(example_string: str) -> None:
    """
     Prints a comparison of three string encodings.
     打印三个字符串编码的比较。
    """
    # print the example string
    print(f'\nExample string: "{example_string}"')
    # for each encoding, print the # of tokens, the token integers, and the token bytes
    # 对于每个编码，打印令牌数、令牌整数和令牌字节
    for encoding_name in ["gpt2", "p50k_base", "cl100k_base"]:
        encoding = tiktoken.get_encoding(encoding_name)
        token_integers = encoding.encode(example_string)
        num_tokens = len(token_integers)
        token_bytes = [encoding.decode_single_token_bytes(token) for token in token_integers]
        print()
        print(f"{encoding_name}: {num_tokens} tokens")
        print(f"token integers: {token_integers}")
        print(f"token bytes: {token_bytes}")
        

In [13]:
compare_encodings("antidisestablishmentarianism")


Example string: "antidisestablishmentarianism"

gpt2: 5 tokens
token integers: [415, 29207, 44390, 3699, 1042]
token bytes: [b'ant', b'idis', b'establishment', b'arian', b'ism']

p50k_base: 5 tokens
token integers: [415, 29207, 44390, 3699, 1042]
token bytes: [b'ant', b'idis', b'establishment', b'arian', b'ism']

cl100k_base: 6 tokens
token integers: [519, 85342, 34500, 479, 8997, 2191]
token bytes: [b'ant', b'idis', b'establish', b'ment', b'arian', b'ism']


In [11]:
compare_encodings("2 + 2 = 4")



Example string: "2 + 2 = 4"

gpt2: 5 tokens
token integers: [17, 1343, 362, 796, 604]
token bytes: [b'2', b' +', b' 2', b' =', b' 4']

p50k_base: 5 tokens
token integers: [17, 1343, 362, 796, 604]
token bytes: [b'2', b' +', b' 2', b' =', b' 4']

cl100k_base: 7 tokens
token integers: [17, 489, 220, 17, 284, 220, 19]
token bytes: [b'2', b' +', b' ', b'2', b' =', b' ', b'4']


In [12]:
compare_encodings("お誕生日おめでとう")



Example string: "お誕生日おめでとう"

gpt2: 14 tokens
token integers: [2515, 232, 45739, 243, 37955, 33768, 98, 2515, 232, 1792, 223, 30640, 30201, 29557]
token bytes: [b'\xe3\x81', b'\x8a', b'\xe8\xaa', b'\x95', b'\xe7\x94\x9f', b'\xe6\x97', b'\xa5', b'\xe3\x81', b'\x8a', b'\xe3\x82', b'\x81', b'\xe3\x81\xa7', b'\xe3\x81\xa8', b'\xe3\x81\x86']

p50k_base: 14 tokens
token integers: [2515, 232, 45739, 243, 37955, 33768, 98, 2515, 232, 1792, 223, 30640, 30201, 29557]
token bytes: [b'\xe3\x81', b'\x8a', b'\xe8\xaa', b'\x95', b'\xe7\x94\x9f', b'\xe6\x97', b'\xa5', b'\xe3\x81', b'\x8a', b'\xe3\x82', b'\x81', b'\xe3\x81\xa7', b'\xe3\x81\xa8', b'\xe3\x81\x86']

cl100k_base: 9 tokens
token integers: [33334, 45918, 243, 21990, 9080, 33334, 62004, 16556, 78699]
token bytes: [b'\xe3\x81\x8a', b'\xe8\xaa', b'\x95', b'\xe7\x94\x9f', b'\xe6\x97\xa5', b'\xe3\x81\x8a', b'\xe3\x82\x81', b'\xe3\x81\xa7', b'\xe3\x81\xa8\xe3\x81\x86']


## 6. Counting tokens for chat API calls
## 6. 计算聊天API调用的令牌数

ChatGPT models like `gpt-3.5-turbo` and `gpt-4` use tokens in the same way as older completions models, but because of their message-based formatting, it's more difficult to count how many tokens will be used by a conversation.
像`gpt-3.5-turbo`和`gpt-4`这样的ChatGPT模型与旧的完成模型一样使用令牌，但由于它们的基于消息的格式，因此很难计算对话中将使用多少令牌。

Below is an example function for counting tokens for messages passed to `gpt-3.5-turbo-0301` or `gpt-4-0314`.
下面是一个示例函数，用于计算传递给`gpt-3.5-turbo-0301`或`gpt-4-0314`的消息的令牌数。
Note that the exact way that tokens are counted from messages may change from model to model. Consider the counts from the function below an estimate, not a timeless guarantee.
请注意，从消息中计算令牌的确切方式可能会因模型而异。将下面函数的计数视为估计值，而不是永恒的保证。

In [14]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301"):
    """Returns the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model == "gpt-3.5-turbo":
        print("Warning: gpt-3.5-turbo may change over time. Returning num tokens assuming gpt-3.5-turbo-0301.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0301")
    elif model == "gpt-4":
        print("Warning: gpt-4 may change over time. Returning num tokens assuming gpt-4-0314.")
        return num_tokens_from_messages(messages, model="gpt-4-0314")
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif model == "gpt-4-0314":
        tokens_per_message = 3
        tokens_per_name = 1
    else:
        raise NotImplementedError(f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens.""")
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens


In [16]:
# let's verify the function above matches the OpenAI API response

import openai

example_messages = [
    {
        "role": "system",
        "content": "You are a helpful, pattern-following assistant that translates corporate jargon into plain English.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "New synergies will help drive top-line growth.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Things working well together will increase revenue.",
    },
    {
        "role": "system",
        "name": "example_user",
        "content": "Let's circle back when we have more bandwidth to touch base on opportunities for increased leverage.",
    },
    {
        "role": "system",
        "name": "example_assistant",
        "content": "Let's talk later when we're less busy about how to do better.",
    },
    {
        "role": "user",
        "content": "This late pivot means we don't have time to boil the ocean for the client deliverable.",
    },
]

for model in ["gpt-3.5-turbo-0301", "gpt-4-0314"]:
    print(model)
    # example token count from the function defined above
    print(f"{num_tokens_from_messages(example_messages, model)} prompt tokens counted by num_tokens_from_messages().")
    # example token count from the OpenAI API
    response = openai.ChatCompletion.create(
        model=model,
        messages=example_messages,
        temperature=0,
        max_tokens=1  # we're only counting input tokens here, so let's not waste tokens on the output
    )
    print(f'{response["usage"]["prompt_tokens"]} prompt tokens counted by the OpenAI API.')
    print()


gpt-3.5-turbo-0301
127 prompt tokens counted by num_tokens_from_messages().
127 prompt tokens counted by the OpenAI API.

gpt-4-0314
129 prompt tokens counted by num_tokens_from_messages().


InvalidRequestError: The model: `gpt-4-0314` does not exist